In [26]:
import pandas as pd
import numpy as np

# combined data - 2 tables

In [27]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [28]:
filename_careers = "careers.xlsx"

careers = pd.read_excel(path_tables + filename_careers,dtype="str")
careers.shape

(9001, 12)

In [29]:
careers[careers["CareerString"].isna()]

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes


In [30]:
filename_leadercareerlink = "leadercareerlink.xlsx"

In [31]:
filename_leaderjoblink = "leaderjoblink.xlsx"

In [32]:
filename_joborglink = "joborglink.xlsx"

# combined data - 1 cleaning/cleaning step 2 - career/

In [33]:
path_cleaning = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/"

In [34]:
subpath_2_1 = "2.1 career_undivided_unparsed_uncoded/"
subpath_2_2 = "2.2 career_divided_unparsed_uncoded/"
subpath_2_3 = "2.3 joborglink/"
subpath_2_4 = "2.4 orgtree/"
subpath_2_5 = "2.5 position/"
subpath_2_6 = "2.6 career_reassembled/"
subpath_2_7 = "2.7 orgtree position & rank/"

In [35]:
# 1.0 orgtree_position_rank

# constructing new file from:
## 2022 북한_기관별_인명록_북한정보포털 게재용
## orgtree table

filein_opr = "1.0 orgtree_position_rank.xlsx"
opr = pd.read_excel(path_cleaning + subpath_2_7 + filein_opr,dtype="str")
opr.shape

(949, 19)

In [36]:
opr.head(2)

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName,Alias_OrgName
0,1,노동당,NaN,1.0.0.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0,NaN,NaN
1,1,노동당,당중앙위원회,1.1.0.0.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0,0,0,NaN,NaN


In [78]:
# orgtree_copy

# copy of orgtree table

filein_ot = "orgtree_copy.xlsx"
ot = pd.read_excel(path_cleaning + subpath_2_7 + filein_ot,dtype="str")
ot.shape

(1406, 19)

In [38]:
# position_rank_jacob_2 - hand-coded

# Jacob's hand_ranked positions of 내각, 정무원

filein_posrank_admin = "position_rank_jacob_2.xlsx"
pr_admin = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_admin,dtype="str")
pr_admin.shape

(1877, 13)

In [39]:
# position_rank_esther - original

# original positions of 노동당, before hand-coding

# filein_posrank_party_orig = "position_rank_esther.xlsx"
# pr_party_orig = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_party_orig,dtype="str")
# pr_party_orig.shape

In [40]:
# position_rank_esther_handcoded

# Esther's hand_ranked positions of 노동당

# filein_posrank_party = "position_rank_esther_handcoded.xlsx"
# pr_party = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_party,dtype="str")
# pr_party.shape

In [41]:
# position_rank_jeongsu_handcoded

# Jeongsu's hand-coded positions of all orgs minus 노동당, 내각, 정무원

# filein_posrank_else = "position_rank_jeongsu.xlsx"
# pr_else = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_else,dtype="str")
# pr_else.shape

# functions: format orgtree

In [42]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

In [43]:
def verify_unique_rows(df):
    
    key_columns = ["PrimaryInstitution","OrgName"]
    print("\nVerifying Unique Rows...")
    print("")
    print("\tDuplicate Rows:",df[df.duplicated(keep=False)].shape)
    print("\tDuplicate Keys:",df[df.duplicated(key_columns,keep=False)].shape)
    print("\tNull Rows     :",df[df["PrimaryInstitution"].isna() & df["OrgName"].isna()].shape)

In [44]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

# 0. opr: format indices & OrgRank

In [45]:
opr.head(2)

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName,Alias_OrgName
0,1,노동당,NaN,1.0.0.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0,NaN,NaN
1,1,노동당,당중앙위원회,1.1.0.0.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0,0,0,NaN,NaN


In [46]:
def update_opr_index(opr,max_level):
    
    # create and clear out existing Level Indicies
    opr["OrgRank"]=np.nan
    opr["L1_Index"]=np.nan
    opr["L2_Index"]=np.nan
    opr["L3_Index"]=np.nan
    opr["L4_Index"]=np.nan
    opr["L5_Index"]=np.nan
    
    # pad PI_Index with missing 0s up to the deepest level (4 or 5?)
    opr["PI_Index"] = opr.apply(lambda x: str(x["PI_Index"]) + ((max_level-1)-str(x["PI_Index"]).count(".")) * ".0",axis=1)
    
    # calculate OrgRank
    opr["OrgRank"] = opr.apply(lambda x: (max_level-1) - str(x["PI_Index"]).count(".0"),axis=1)
    
    # populate Level Indices using PI_Index
    opr.loc[opr["PI_Index"].notna(),"L1_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[0],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L2_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[1],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L3_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[2],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L4_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[3],axis=1)
    opr.loc[opr["PI_Index"].notna(),"L5_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[4],axis=1)

    # sort by Level1, Level2, Level3
    sort_columns = ["L1_Index","L2_Index","L3_Index","L4_Index","L5_Index"]
    #opr.L1_Index.astype(int,errors="ignore")
    #opr.L2_Index.astype(int,errors="ignore")
    #opr.L3_Index.astype(int,errors="ignore")
    opr.sort_values(sort_columns,inplace=True)
    
    return opr

In [47]:
opr = update_opr_index(opr,5)

In [49]:
# export opr

# fileout_opr = "1.0 orgtree_position_rank.xlsx"
# opr.to_excel(path_cleaning + subpath_2_7 + fileout_opr,index=False)

# 0. pr_admin - clean file, add LinkToNext_

In [50]:
# position_rank_esther_handcoded: restore OrgName from position_rank_esther
# Esther changed a few OrgNames; I will restore the original OrgName, but save Esther's changes

In [51]:
pr_admin.head(2)

,CareerString,CareerSubstring,OrgString,CareerStartYear,PrimaryInstitution,OrgName,Position,InsideGov,IsElected,PositionRank,Notes,Change_OrgName,Change_Position
0,2009. 내각 225국 국장,NaN,내각 225국,2009,내각,225국,국장,1,NaN,1,NaN,NaN,NaN
1,내각 225부 부장,NaN,내각 225부,2009,내각,225국,부장,1,NaN,1,NaN,NaN,국장


In [52]:
pr_admin[pr_admin.Change_OrgName.notna()].shape

(9, 13)

In [53]:
# add LinkToNext_ fields to pr_party2
ot.head(2)

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
0,0,4.15문화창작단,NaN,NaN,NaN,Current,Current,Current,NaN
1,0,6.15공동선언실천남북공동위원회북측위원회,교직원분과,NaN,NaN,Current,Current,Current,NaN


In [54]:
key_columns2 = ["PrimaryInstitution","OrgName"]
# org_columns = key_columns2 + [c for c in list(ot.columns) if "LinkToNext_" in c]
org_columns = key_columns2 + ["LinkToNext_Year"]
# print(org_columns)
pr_admin2 = pr_admin.merge(ot[org_columns],on=key_columns2,how="inner",indicator=False)
# merge_results(pr_admin2)

In [55]:
# prep columns to merge with opr
pr_admin2.rename(columns={"PositionRank":"PositionRank_manual"},inplace=True)

In [56]:
pr_admin2["PositionRank_3P"]=np.nan

In [57]:
pr_admin2.columns

Index(['CareerString', 'CareerSubstring', 'OrgString', 'CareerStartYear',
       'PrimaryInstitution', 'OrgName', 'Position', 'InsideGov', 'IsElected',
       'PositionRank_manual', 'Notes', 'Change_OrgName', 'Change_Position',
       'LinkToNext_Year', 'PositionRank_3P'],
      dtype='object')

In [58]:
reorder_columns = ['CareerString', 'CareerSubstring', 'CareerStartYear','InsideGov', 
       'PrimaryInstitution', 'OrgName', 'LinkToNext_Year','Position', 'IsElected',
       'PositionRank_manual',  'PositionRank_3P', 'Change_OrgName', 'Change_Position','Notes']
pr_admin2 = pr_admin2[reorder_columns]

In [59]:
pr_admin2.head(2)

,CareerString,CareerSubstring,CareerStartYear,InsideGov,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_OrgName,Change_Position,Notes
0,2009. 내각 225국 국장,NaN,2009,1,내각,225국,2015,국장,NaN,1,NaN,NaN,NaN,NaN
1,내각 225부 부장,NaN,2009,1,내각,225국,2015,부장,NaN,1,NaN,NaN,국장,NaN


# 1. orgtree_position_rank: update 내각, 정무원

In [60]:
# align opr with pr_admin2

In [61]:
pr_admin2.head(2)

,CareerString,CareerSubstring,CareerStartYear,InsideGov,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_OrgName,Change_Position,Notes
0,2009. 내각 225국 국장,NaN,2009,1,내각,225국,2015,국장,NaN,1,NaN,NaN,NaN,NaN
1,내각 225부 부장,NaN,2009,1,내각,225국,2015,부장,NaN,1,NaN,NaN,국장,NaN


In [62]:
# recursively - 

## manually update orgtree_position_rank
## open and compare with pr_party2

In [80]:
# 1.0 orgtree_position_rank

# constructing new file from:
## 2022 북한_기관별_인명록_북한정보포털 게재용
## orgtree table

filein_opr = "1.0 orgtree_position_rank.xlsx"
opr = pd.read_excel(path_cleaning + subpath_2_7 + filein_opr,dtype="str")
opr.shape

(1285, 19)

In [81]:
opr.head(2)

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName,Alias_OrgName
0,1,노동당,NaN,1.0.0.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0,NaN,NaN
1,1,노동당,당중앙위원회,1.1.0.0.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0,0,0,NaN,NaN


In [82]:
opr = update_opr_index(opr,5)

In [83]:
# export opr

fileout_opr = "1.0 orgtree_position_rank.xlsx"
opr.to_excel(path_cleaning + subpath_2_7 + fileout_opr,index=False)

In [84]:
pr_admin2.columns

Index(['CareerString', 'CareerSubstring', 'CareerStartYear', 'InsideGov',
       'PrimaryInstitution', 'OrgName', 'LinkToNext_Year', 'Position',
       'IsElected', 'PositionRank_manual', 'PositionRank_3P', 'Change_OrgName',
       'Change_Position', 'Notes'],
      dtype='object')

In [85]:
opr.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'PI_Index', 'OrgRank',
       'P1', 'P2', 'P3', 'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year',
       'Notes', 'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index',
       'Change_OrgName', 'Alias_OrgName'],
      dtype='object')

In [86]:
key_columns2 = ["PrimaryInstitution","OrgName"]
pr_admin3 = pr_admin2.merge(opr,on=key_columns2,how="left",indicator=True,suffixes=("","_opr"))
merge_results(pr_admin3)


Merge Results...

	shape     : (1878, 32)
	left_only : (300, 32)
	both      : (1578, 32)
	right_only: (0, 32)


In [87]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# select_columns = ["OrgName","PI_Index","OrgRank","Position","PositionRank_manual","P1","P2","P3","Notes","Notes_opr"] 
# pr_admin3.loc[pr_admin3["_merge"]=="left_only",select_columns]

In [88]:
pr_admin3[pr_admin3["_merge"]=="left_only"].head(3)

,CareerString,CareerSubstring,CareerStartYear,InsideGov,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_OrgName,Change_Position,Notes,InsideGov_opr,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName_opr,Alias_OrgName,_merge
2,1975.10 농업위원회 간척지 설계사업소 기사장,NaN,1975,1,내각,간석지건설지도국_간석지설계사업소,Current,기사장,NaN,1,NaN,NaN,NaN,"Currently, 사업소 지배인; 연구원 기사장",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,농업위원회 간척지 설계사업소 기사장,NaN,1975,1,내각,간석지건설지도국_간석지설계사업소,Current,기사장,NaN,1,NaN,NaN,NaN,"Currently, 사업소 지배인; 연구원 기사장",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
16,2003. 8 건설건재공업성 과학기술국장,NaN,2003,1,내각,건설건재공업성_과학기술국,Current,국장,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


### Three Cases of  OrgName in NK elite career data

- Case 0: OrgName in 기관별인명록: code these from 1-199
- Case 1: OrgName not in 기관별인명록, but contained in data AND LinkToNext_Year not current: code as 500+
- Case 2: OrgName not in 기관별인명록, but contained in data AND LinkToNext_Year is current: code as 200+

### Case 0: OrgName in 기관별인명록

- Update 1.0 orgtree_position_rank with OrgName in 기관별인명록
- Code these from 1-199
- Run above routine # 1. orgtree_position_rank: update 내각, 정무원
    - to update opr & pr_admin3
- Proceed to Cases 1 & 2

### Case 1: LinkToNext_Year not current - code as 500+

In [89]:
pr_admin3[(pr_admin3["LinkToNext_Year"]!="Current") & (pr_admin3["_merge"]=="left_only")].shape

(0, 32)

In [90]:
ot[~(ot["LinkToNext_Year"]=="Current") & (ot["PrimaryInstitution"].isin(["정무원","내각"]))].shape

(0, 19)

In [91]:
pr_admin3[(pr_admin3["LinkToNext_Year"]!="Current") & (pr_admin3["_merge"]=="left_only")]

,CareerString,CareerSubstring,CareerStartYear,InsideGov,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_OrgName,Change_Position,Notes,InsideGov_opr,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName_opr,Alias_OrgName,_merge


### Case 2: Level 3 or deeper (not found in 기관별인명록) - code as 200+

In [92]:
pr_admin3[pr_admin3["_merge"]=="left_only"].shape

(300, 32)

In [93]:
show_columns = ["OrgName","Position","Notes","Change_OrgName","_merge"]
pr_admin3.loc[pr_admin3["_merge"]=="left_only",show_columns]

,OrgName,Position,Notes,Change_OrgName,_merge
2,간석지건설지도국_간석지설계사업소,기사장,"Currently, 사업소 지배인; 연구원 기사장",NaN,left_only
3,간석지건설지도국_간석지설계사업소,기사장,"Currently, 사업소 지배인; 연구원 기사장",NaN,left_only
16,건설건재공업성_과학기술국,국장,NaN,NaN,left_only
64,공보위원회,위원장,교육위원회_고등교육성?,NaN,left_only
97,교육위원회_고등교육성_김일성종합대학,총장,교육위원회_고등교육성?,NaN,left_only
98,교육위원회_고등교육성_김일성종합대학,총장,교육위원회_고등교육성?,NaN,left_only
99,교육위원회_고등교육성_김일성종합대학,총장,교육위원회_고등교육성?,NaN,left_only
100,교육위원회_고등교육성_평양교원대학,총장,교육위원회_고등교육성?,NaN,left_only
101,교육위원회_고등교육성_평양국제관계대학,교수,교육위원회_고등교육성?,NaN,left_only
102,교육위원회_보통교육성,상,교육위원회_고등교육성?,NaN,left_only


In [94]:
# prep to add to opr
opr.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'PI_Index', 'OrgRank',
       'P1', 'P2', 'P3', 'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year',
       'Notes', 'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index',
       'Change_OrgName', 'Alias_OrgName'],
      dtype='object')

In [95]:
pr_admin3[opr.columns].head(2)

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName,Alias_OrgName
0,1,내각,225국,nan.0.0.0.0,0.0,NaN,NaN,NaN,노동당,당중앙위원회_문화교류국,2015,NaN,nan,0,0,0,0,NaN,NaN
1,1,내각,225국,nan.0.0.0.0,0.0,NaN,NaN,NaN,노동당,당중앙위원회_문화교류국,2015,NaN,nan,0,0,0,0,NaN,NaN


In [97]:
pr_admin3_left = pr_admin3.loc[pr_admin3["_merge"]=="left_only",opr.columns]

In [98]:
pr_admin3_left.shape

(300, 19)

In [99]:
pr_admin3_left.drop_duplicates(inplace=True)
pr_admin3_left.shape

(93, 19)

In [100]:
pr_admin3_left

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName,Alias_OrgName
2,1,내각,간석지건설지도국_간석지설계사업소,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,"Currently, 사업소 지배인; 연구원 기사장",NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,1,내각,건설건재공업성_과학기술국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,1,내각,공보위원회,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,교육위원회_고등교육성?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,0,내각,교육위원회_고등교육성_김일성종합대학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,교육위원회_고등교육성?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,0,내각,교육위원회_고등교육성_평양교원대학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,교육위원회_고등교육성?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,0,내각,교육위원회_고등교육성_평양국제관계대학,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,교육위원회_고등교육성?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,1,내각,교육위원회_보통교육성,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,교육위원회_고등교육성?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,1,내각,국가관광총국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,_X국,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,1,내각,국가관광총국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,1,내각,국가비상방역위원회,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# export pr_admin3_left

fileout_pr_admin3_left = "pr_admin3_left_use and delete.xlsx"
pr_admin3_left.to_excel(path_cleaning + subpath_2_7 + fileout_pr_admin3_left,index=False)

# 2. pr_admin3: update PositionRank_3P and export

In [306]:
# once aligned, copy pr_party2 hand-coded PositionRank into opr

In [102]:
pr_admin.shape

(1877, 13)

In [103]:
pr_admin2.shape

(1877, 14)

In [104]:
pr_admin3.shape

(1878, 32)

In [105]:
pr_admin3.head(10)

,CareerString,CareerSubstring,CareerStartYear,InsideGov,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_OrgName,Change_Position,Notes,InsideGov_opr,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName_opr,Alias_OrgName,_merge
0,2009. 내각 225국 국장,NaN,2009,1,내각,225국,2015,국장,NaN,1,NaN,NaN,NaN,NaN,1,nan.0.0.0.0,0.0,NaN,NaN,NaN,노동당,당중앙위원회_문화교류국,2015,NaN,nan,0,0,0,0,NaN,NaN,both
1,내각 225부 부장,NaN,2009,1,내각,225국,2015,부장,NaN,1,NaN,NaN,국장,NaN,1,nan.0.0.0.0,0.0,NaN,NaN,NaN,노동당,당중앙위원회_문화교류국,2015,NaN,nan,0,0,0,0,NaN,NaN,both
2,1975.10 농업위원회 간척지 설계사업소 기사장,NaN,1975,1,내각,간석지건설지도국_간석지설계사업소,Current,기사장,NaN,1,NaN,NaN,NaN,"Currently, 사업소 지배인; 연구원 기사장",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,농업위원회 간척지 설계사업소 기사장,NaN,1975,1,내각,간석지건설지도국_간석지설계사업소,Current,기사장,NaN,1,NaN,NaN,NaN,"Currently, 사업소 지배인; 연구원 기사장",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,1998. 7 건설건재공업성 부상,NaN,1998,1,내각,건설건재공업성,Current,부상,NaN,2,NaN,NaN,NaN,NaN,1,3.12.0.0.0,1.0,상,부상,NaN,Current,Current,Current,NaN,3,12,0,0,0,NaN,NaN,both
5,건설건재공업성 부상,NaN,1998,1,내각,건설건재공업성,Current,부상,NaN,2,NaN,NaN,NaN,NaN,1,3.12.0.0.0,1.0,상,부상,NaN,Current,Current,Current,NaN,3,12,0,0,0,NaN,NaN,both
6,건설건재공업성 과학기술국장,NaN,2003,1,내각,건설건재공업성,Current,국장,NaN,1,NaN,건설건재공업성_과학기술국,NaN,NaN,1,3.12.0.0.0,1.0,상,부상,NaN,Current,Current,Current,NaN,3,12,0,0,0,NaN,NaN,both
7,2005. 건설건재공업성 부상,NaN,2005,1,내각,건설건재공업성,Current,부상,NaN,2,NaN,NaN,NaN,NaN,1,3.12.0.0.0,1.0,상,부상,NaN,Current,Current,Current,NaN,3,12,0,0,0,NaN,NaN,both
8,건설건재공업성 부상,NaN,2005,1,내각,건설건재공업성,Current,부상,NaN,2,NaN,NaN,NaN,NaN,1,3.12.0.0.0,1.0,상,부상,NaN,Current,Current,Current,NaN,3,12,0,0,0,NaN,NaN,both
9,2005. 3 건설건재공업상,NaN,2005,1,내각,건설건재공업성,Current,상,NaN,1,NaN,NaN,NaN,NaN,1,3.12.0.0.0,1.0,상,부상,NaN,Current,Current,Current,NaN,3,12,0,0,0,NaN,NaN,both


In [106]:
def return_position_rank(x):
    
    # create list of positions in P1, P2, P3
    p_list = str(x["P1"]).split(",")+str(x["P2"]).split(",")+str(x["P3"]).split(",")
    # print(p_list)
    
    # remove "nan"
    p_list = [item.strip() for item in p_list if item.strip()!="nan"]
    # print(p_list)
    
    # list index of Position in p_list
    position_rank = np.nan
    
    if x["Position"] in p_list:
        position_rank = p_list.index(x["Position"]) + 1
    
    
    return position_rank

In [107]:
pr_admin3.apply(lambda x: return_position_rank(x),axis=1)

0       NaN
1       NaN
2       NaN
3       NaN
4       2.0
5       2.0
6       NaN
7       2.0
8       2.0
9       1.0
10      1.0
11      1.0
12      1.0
13      1.0
14      1.0
15      1.0
16      NaN
17      NaN
18      1.0
19      1.0
20      2.0
21      2.0
22      2.0
23      1.0
24      1.0
25      1.0
26      1.0
27      2.0
28      1.0
29      1.0
30      1.0
31      1.0
32      1.0
33      1.0
34      1.0
35      NaN
36      NaN
37      NaN
38      NaN
39      NaN
40      NaN
41      NaN
42      NaN
43      NaN
44      NaN
45      NaN
46      NaN
47      NaN
48      NaN
49      NaN
50      NaN
51      NaN
52      NaN
53      NaN
54      NaN
55      NaN
56      NaN
57      NaN
58      NaN
59      NaN
60      NaN
61      NaN
62      NaN
63      NaN
64      NaN
65      NaN
66      NaN
67      NaN
68      NaN
69      NaN
70      NaN
71      NaN
72      NaN
73      NaN
74      NaN
75      NaN
76      NaN
77      NaN
78      NaN
79      NaN
80      NaN
81      NaN
82      2.0
83  

In [108]:
pr_admin3["PositionRank_3P"] = pr_admin3.apply(lambda x: return_position_rank(x),axis=1)

In [109]:
# check
pr_admin3[(pr_admin3["Position"]=="위원") & (pr_admin3["OrgName"]=="당중앙위원회")]

,CareerString,CareerSubstring,CareerStartYear,InsideGov,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,Change_OrgName,Change_Position,Notes,InsideGov_opr,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName_opr,Alias_OrgName,_merge


In [110]:
pr_admin3.columns

Index(['CareerString', 'CareerSubstring', 'CareerStartYear', 'InsideGov',
       'PrimaryInstitution', 'OrgName', 'LinkToNext_Year', 'Position',
       'IsElected', 'PositionRank_manual', 'PositionRank_3P', 'Change_OrgName',
       'Change_Position', 'Notes', 'InsideGov_opr', 'PI_Index', 'OrgRank',
       'P1', 'P2', 'P3', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year_opr', 'Notes_opr', 'L1_Index', 'L2_Index', 'L3_Index',
       'L4_Index', 'L5_Index', 'Change_OrgName_opr', 'Alias_OrgName',
       '_merge'],
      dtype='object')

In [373]:
# export pr_admin3 for Esther & Jeongsu to review

export_columns = ["InsideGov","PrimaryInstitution","OrgName","OrgRank","LinkToNext_Year","Position","IsElected","PositionRank_manual","PositionRank_3P","P1","P2","P3","Change_OrgName","Change_Position","Notes"]
pr_admin3[export_columns].to_excel(path_cleaning + subpath_2_7 + "position_rank_esther_two_ranks.xlsx",index=False)

# 3. match career & pr_party3 and resolve null Position_3P

#### Resolve. Validation values for resolving null Position_3P

1. OrgName & Position ok. Add Position to P1
2. OrgName & Position ok. Add Position to P2
3. OrgName & Position ok. Add Position to P3
4. Change_OrgName and/or Change_Position
5. Uncertain OrgName
6. Uncertain Position
7. NotJob
8. Multiple CareerSubstring
9. Other - see Notes

In [374]:
careers.head(2)

,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes
0,1996. 3 외교부 10국장,NaN,True,1,1996,3,NaN,외교부,정무원,외교부,국장,NaN
1,유엔 주재 대표부 1등서기관,1975.10,True,1,1975,10,NaN,유엔 주재 대표부,내각,외무성_유엔대표부,1등서기관,NaN


In [375]:
careers_columns = ["CareerString","CareerStartYear","MultipleSubstrings","CareerSubstring","IsJob","PrimaryInstitution","OrgName","Position"]
careers[careers_columns].head(2)

,CareerString,CareerStartYear,MultipleSubstrings,CareerSubstring,IsJob,PrimaryInstitution,OrgName,Position
0,1996. 3 외교부 10국장,1996,1,NaN,True,정무원,외교부,국장
1,유엔 주재 대표부 1등서기관,1975,1,NaN,True,내각,외무성_유엔대표부,1등서기관


In [376]:
pr_party4.head(2)

,InsideGov,PrimaryInstitution,OrgName,Position,IsElected,OrgLevel,PositionRank_manual,Notes,Change_OrgName,Change_Position,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,PositionRank_3P,InsideGov_opr,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI_opr,LinkToNext_Org_opr,LinkToNext_Year_opr,Notes_opr,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Change_OrgName_opr,Alias_OrgName,_merge
0,1,노동당,Uncertain,부부장,NaN,NaN,?,NaN,NaN,NaN,Current,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,1,노동당,당대표자회_집행부,성원,NaN,NaN,NaN,"NotOrg - 대표자회는 임시 회의, '임시 당대회'",NaN,NaN,Current,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [377]:
pr_party4_incomplete = pr_party4[pr_party4["PositionRank_3P"].isna()]
pr_party4_incomplete.shape

(147, 32)

In [378]:
pr_match_columns = ["PrimaryInstitution","OrgName","LinkToNext_Year","Position","IsElected","PositionRank_manual","PositionRank_3P","P1","P2","P3","Change_OrgName","Change_Position","Notes"]
pr_party4_incomplete = pr_party4_incomplete[pr_match_columns]
pr_party4_incomplete["Resolution"] = np.nan

In [379]:
pr_order_columns = ["PrimaryInstitution","OrgName","LinkToNext_Year","Position","IsElected","PositionRank_manual","PositionRank_3P","P1","P2","P3","Resolution","Change_OrgName","Change_Position","Notes"]
pr_party4_incomplete = pr_party4_incomplete[pr_order_columns]
pr_party4_incomplete.head(2)

,PrimaryInstitution,OrgName,LinkToNext_Year,Position,IsElected,PositionRank_manual,PositionRank_3P,P1,P2,P3,Resolution,Change_OrgName,Change_Position,Notes
0,노동당,Uncertain,Current,부부장,NaN,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,노동당,당대표자회_집행부,Current,성원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NotOrg - 대표자회는 임시 회의, '임시 당대회'"


In [380]:
key_columns3 = ["PrimaryInstitution","OrgName","Position"]
career_pos_mismatch = careers[careers_columns].merge(pr_party4_incomplete,on=key_columns3,how="inner",indicator=False)
# merge_results(career_pos_mismatch)

In [381]:
# career_pos_mismatch[career_pos_mismatch["_merge"]=="right_only"]

In [382]:
career_pos_mismatch.head(10)

,CareerString,CareerStartYear,MultipleSubstrings,CareerSubstring,IsJob,PrimaryInstitution,OrgName,Position,LinkToNext_Year,IsElected,PositionRank_manual,PositionRank_3P,P1,P2,P3,Resolution,Change_OrgName,Change_Position,Notes
0,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",1951,4,당 과학교육부,True,노동당,당중앙위원회_과학교육부,uncertain,Current,NaN,?,NaN,부장,제1부부장,NaN,NaN,NaN,NaN,NaN
1,1975. 당 국제부 근무,1975,1,NaN,True,노동당,당중앙위원회_국제부,Uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
2,1975. 당 국제부 근무,1975,1,NaN,False,노동당,당중앙위원회_국제부,Uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
3,당 국제부 근무,1975,1,NaN,True,노동당,당중앙위원회_국제부,Uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
4,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",1951,4,당 조직지도부,True,노동당,당중앙위원회_조직지도부,uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
5,1978. 당 통일전선부 근무,1978,1,NaN,False,노동당,당중앙위원회_통일전선부,Uncertain,Current,NaN,?,NaN,부장,부부장,NaN,NaN,NaN,NaN,NaN
6,1983. 당 통일전선부 근무,1983,1,NaN,True,노동당,당중앙위원회_통일전선부,Uncertain,Current,NaN,?,NaN,부장,부부장,NaN,NaN,NaN,NaN,NaN
7,1964. 당 고급당학교 강좌장,1964,1,NaN,True,노동당,당중앙위원회_김일성고급당학교,강좌장,Current,NaN,2,NaN,총장,부총장,NaN,NaN,NaN,NaN,NaN
8,당 고급당학교 강좌장,1964,1,NaN,True,노동당,당중앙위원회_김일성고급당학교,강좌장,Current,NaN,2,NaN,총장,부총장,NaN,NaN,NaN,NaN,NaN
9,1999. 9 당 중앙위 검열위원장,1999,1,NaN,True,노동당,당중앙위원회,위원장,Current,1,3,NaN,NaN,NaN,"위원, 후보위원",NaN,NaN,NaN,NaN


In [383]:
career_pos_mismatch[career_pos_mismatch["Position"].str.lower()=="uncertain"]

,CareerString,CareerStartYear,MultipleSubstrings,CareerSubstring,IsJob,PrimaryInstitution,OrgName,Position,LinkToNext_Year,IsElected,PositionRank_manual,PositionRank_3P,P1,P2,P3,Resolution,Change_OrgName,Change_Position,Notes
0,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",1951,4,당 과학교육부,True,노동당,당중앙위원회_과학교육부,uncertain,Current,NaN,?,NaN,부장,제1부부장,NaN,NaN,NaN,NaN,NaN
1,1975. 당 국제부 근무,1975,1,NaN,True,노동당,당중앙위원회_국제부,Uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
2,1975. 당 국제부 근무,1975,1,NaN,False,노동당,당중앙위원회_국제부,Uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
3,당 국제부 근무,1975,1,NaN,True,노동당,당중앙위원회_국제부,Uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
4,"1951.~57. 루마니아 유학 연도미상 당 조직지도부, 과학교육부, 기계공업부 등...",1951,4,당 조직지도부,True,노동당,당중앙위원회_조직지도부,uncertain,Current,NaN,?,NaN,부장,"제1부부장, 부부장",NaN,NaN,NaN,NaN,NaN
5,1978. 당 통일전선부 근무,1978,1,NaN,False,노동당,당중앙위원회_통일전선부,Uncertain,Current,NaN,?,NaN,부장,부부장,NaN,NaN,NaN,NaN,NaN
6,1983. 당 통일전선부 근무,1983,1,NaN,True,노동당,당중앙위원회_통일전선부,Uncertain,Current,NaN,?,NaN,부장,부부장,NaN,NaN,NaN,NaN,NaN


In [384]:
career_pos_mismatch.shape

(496, 19)

In [385]:
# export and edit

# career_pos_mismatch.to_excel(path_cleaning + subpath_2_7 + "2.0_career_pos_mismatch_0.xlsx",index=False)